In [27]:
import cv2 as cv
import imutils
import numpy as np
frames=8
min_size=100
movement=10
cap=cv.VideoCapture(5)
cap.release()
cap=cv.VideoCapture(0)
firstf=None
secondf=None
font=cv.FONT_HERSHEY_COMPLEX
delay=0
movementc=0
while True:
    flag=False
    ret,frame=cap.read()
    text="Nobody"
    if not ret:
        print("error")
        continue
    frame=imutils.resize(frame,width=500)
    grey=cv.cvtColor(frame,cv.COLOR_BGR2GRAY)
    grey=cv.GaussianBlur(grey, (21,21),0)
    if firstf is None:
        firstf=grey
        
    delay=delay+1
    
    if delay>frames:
        delay=0
        firstf=secondf
    secondf=grey
    
    delta=cv.absdiff(firstf,secondf)
    thresh=cv.threshold(delta,25,255,cv.THRESH_BINARY)[1]
    thresh=cv.dilate(thresh, None, iterations=5)
    cont,_=cv.findContours(thresh.copy(),cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    for c in cont:
        (x,y,w,h)=cv.boundingRect(c)
        if cv.contourArea(c)>min_size:
            flag=True
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    if flag==True:
        m_flag=True
        movementc=movement
    if movementc>0:
        text="Movement detected: "+str(movementc)
        movementc=-1
    else:
        text="No movement"
    cv.putText(frame, str(text),(10,30),font,0.75,(255,0,0),2, cv.LINE_AA)
    #cv.imshow("frame",frame)
    #cv.imshow("delta",delta)
    delta=cv.cvtColor(delta,cv.COLOR_GRAY2BGR)
    cv.imshow("frame",np.hstack((delta,frame)))
    ch=cv.waitKey(1)
    if ch & 0xFF ==ord('q'):
        break
cv.waitKey(0)
cv.destroyAllWindows()
cap.release()